In [1]:
%matplotlib qt
import os
import numpy as np
import pandas as pd
import glob
import os
import math
import matplotlib
from matplotlib import pyplot as plt, dates
import scipy
from scipy import optimize
import sklearn
import requests
from io import StringIO
import sys
import csv   
from csv import writer
import time
import shutil
import pickle
import seaborn as sns
import datetime as dt

In [2]:
def my_callback_function(state):      
    global low_limit, high_limit,hot_complaint,cold_complaint, occupied_time
    if api.exchange.warmup_flag(state):
        return
    envNum = api.exchange.current_environment_num(state)
    if envNum == 3: # if sizing done 
        tSetHtg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetHtg")
        tSetClg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetClg")
        tIn_office_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Office")
        tIn_lobby_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Lobby")
        htg_hndl = api.exchange.get_meter_handle(state,"Heating:Electricity")
        clg_hndl = api.exchange.get_meter_handle(state,"Cooling:Electricity")

        # exchange information with EnergyPlus
        month = api.exchange.month(state) 
        time_of_day = api.exchange.current_time(state)
#         hours = api.exchange.hour(state)
        day_of_week = api.exchange.day_of_week(state)
        day_of_year = api.exchange.day_of_year(state)
        day_of_month = api.exchange.day_of_month(state)
        tIn_office = api.exchange.get_variable_value(state,tIn_office_hndl)
        tIn_lobby = api.exchange.get_variable_value(state,tIn_lobby_hndl)
        heating_meter_reading = api.exchange.get_meter_value(state,htg_hndl)
        cooling_meter_reading = api.exchange.get_meter_value(state,clg_hndl)
        
        ## occupant complaint model
        cold_complaint = 0
        hot_complaint = 0 
        if (time_of_day > 8) and (time_of_day < 17) and (day_of_week > 1): # occupied periods
            occupied_time = occupied_time + 1
            if (tIn_office < low_limit-0.1) or (tIn_lobby < low_limit-0.1):
                cold_complaint = 1
            elif (tIn_office > high_limit+0.1) or (tIn_lobby > high_limit+0.1):
                hot_complaint = 1
        
        ## actuate DLC parameters
        api.exchange.set_actuator_value(state, tSetHtg_hndl, 18)
        api.exchange.set_actuator_value(state, tSetClg_hndl, 27)


        # bank operates on weekdays and on Saturdays
        if (day_of_week > 1):
            if (time_of_day > 7 and time_of_day < 17):
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21)
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24)

        # time of use rates apply to weekdays only
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - ph_dur) and (time_of_day < 7): # preheat winter off-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 18 + ph_mag)
            if (time_of_day >= 7) and (time_of_day < 11): # setback winter on-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21 - sb_mag)

        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - pc1_dur) and (time_of_day < 7): # precool summer off-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 27 - pc1_mag)
            if (time_of_day >= 7) and (time_of_day < 11 - pc2_dur): # setup summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + su1_mag)
            if (time_of_day >= 11 - pc2_dur) and (time_of_day < 11): # precool summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 - pc2_mag)
            if (time_of_day >= 11) and (time_of_day <= 17): # setup summer on-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + su2_mag)

        # rate calculator
        cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*8.7/100
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7) and (time_of_day <= 11):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100
            elif (time_of_day > 11) and (time_of_day <= 17):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
            elif (time_of_day > 17) and (time_of_day < 19): 
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100

        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7) and (time_of_day <= 11):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
            elif (time_of_day > 11) and (time_of_day <= 17):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100
            elif (time_of_day > 17) and (time_of_day < 19): 
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
                
        electricity.append(cost) 
        discomfort.append(cold_complaint + hot_complaint)
        

In [ ]:
counter = 0

def wrapper(parameters):
    global counter,ph_mag,ph_dur,sb_mag, pc1_mag, pc1_dur, su1_mag, pc2_mag, pc2_dur, su2_mag, api, electricity, discomfort, low_limit, high_limit, epwFile,idfFile 
    
    electricity = []
    discomfort = []
    
    ph_mag = parameters[0]
    ph_dur = parameters[1]
    sb_mag = parameters[2]
    
    pc1_mag = parameters[3]
    pc1_dur = parameters[4]
    su1_mag = parameters[5]
    
    pc2_mag = parameters[6]
    pc2_dur = parameters[7]
    su2_mag = parameters[8]  
              
    sys.path.insert(0, 'C:\EnergyPlusV23-1-0')  # add E-Plus directory to path to be able to import API
    from pyenergyplus.api import EnergyPlusAPI #import EnergyPlus library
    api = EnergyPlusAPI()
    state = api.state_manager.new_state() 
    api.runtime.clear_callbacks()
    api.functional.clear_callbacks()
    
    # energyplus model calling point, callback function
    api.runtime.callback_begin_system_timestep_before_predictor(state,my_callback_function)
    # run EPlus
    cmd_args = ['-w',epwFile, '-d', 'out','-x',idfFile]
    api.runtime.run_energyplus(state,cmd_args)
    api.state_manager.delete_state(state)

    counter = counter + 1
    print(counter)
    
#     obj = sum(electricity) + sum(electricity)*(sum(discomfort) > 1000)
    obj = sum(electricity) + sum(electricity)*(((sum(discomfort)/len(discomfort))*10)**2)

    with open('document.csv', 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow([sum(electricity),sum(discomfort)/4, obj, ph_mag,ph_dur,sb_mag, pc1_mag, pc1_dur, su1_mag, pc2_mag, pc2_dur, su2_mag])
        f_object.close()
    return obj

In [ ]:
start = time.time()
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
active_cases = scenarios_info[scenarios_info.active == 1]
bounds = [[0,6],[0,3],[0,3],[0,6],[0,3],[0,3],[0,3],[0,3],[0,3]] 
for i, row in active_cases.iterrows():
    
    idfFile = "Models_heatpump/"+row.idf_file
    epwFile = row.epw_file
    output_dir = "Results/Scenario_{}".format(row.ID)
    [low_limit, high_limit] = eval(row.comfort_range)
    occupied_time = 0
    results = scipy.optimize.differential_evolution(wrapper,bounds,maxiter=6, popsize=8, polish=False)
    # save results
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    shutil.copy("out/eplusout.csv", output_dir+"/eplusout.csv")
    shutil.copy("document.csv", output_dir+"/document.csv")
    # save results variable: the output of optimization
    with open(output_dir+"/opt_results.pkl", 'wb') as file:
        pickle.dump(results, file) 
    # erase document after copying
    os.remove("document.csv")
end = time.time()
print((end - start)/3600)

# Run with optimal parameters

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
active_cases = scenarios_info[scenarios_info.active == 1]
active_cases

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
active_cases = scenarios_info[scenarios_info.active == 1]
for i, row in active_cases.iterrows():     
    output_dir = "Results/Scenario_{}".format(row.ID)
    idfFile = "Models_heatpump/"+row["idf_file"]
    [low_limit, high_limit] = eval(row["comfort_range"])
    dlc_type = row["description"]
    occupied_time = 0
    if dlc_type == "GA":
        with open('Results\Scenario_{}\opt_results.pkl'.format(row.ID), 'rb') as f:
                results = pickle.load(f)
        parameters = results.x
    elif dlc_type == "no_dlc":
        parameters = [0,0,0,0,0,0,0,0,0]
    elif dlc_type == "dlc_1":
        parameters = [2,1,1,2,1,1,1,1,1]
    elif dlc_type == "dlc_2":
        parameters = [4,2,2,4,2,2,2,2,2]
    elif dlc_type == "dlc_3":
        parameters = [6,3,3,6,3,3,3,3,3]
    elif dlc_type == "dlc_4":
        parameters = [0,0,1,0,0,1,0,0,1]
    elif dlc_type == "dlc_5":
        parameters = [0,0,2,0,0,2,0,0,2]
    elif dlc_type == "dlc_6":
        parameters = [0,0,3,0,0,3,0,0,3]
            

    ph_mag = parameters[0]
    ph_dur = parameters[1]
    sb_mag = parameters[2]

    pc1_mag = parameters[3]
    pc1_dur = parameters[4]
    su1_mag = parameters[5]

    pc2_mag = parameters[6]
    pc2_dur = parameters[7]
    su2_mag = parameters[8] 
    print(ph_mag, ph_dur, sb_mag, pc1_mag, pc1_dur, su1_mag, pc2_mag, pc2_dur, su2_mag)

    electricity = []
    discomfort = []
    time_list = []
    
    print(output_dir, idfFile, low_limit, high_limit)

    sys.path.insert(0, 'C:\EnergyPlusV23-1-0')  # add E-Plus directory to path to be able to import API
    from pyenergyplus.api import EnergyPlusAPI #import EnergyPlus library
    api = EnergyPlusAPI()
    state = api.state_manager.new_state() 
    # energyplus model calling point, callback function
    api.runtime.callback_begin_system_timestep_before_predictor(state,my_callback_function)
    # run EPlus
    epwFile = 'CAN_ON_TORONTO-INTL-A_6158731_CWEC.epw'
    # -x short form to run expandobjects for HVACtemplates. see EnergyPlusEssentials.pdf p16
    cmd_args = ['-w',epwFile, '-d', 'out','-x',idfFile]
    api.runtime.run_energyplus(state,cmd_args)
    api.state_manager.delete_state(state)

    electricity_cost = sum(electricity)
    discomfort_cost = sum(discomfort)/4
    obj = sum(electricity) + sum(electricity)*(((sum(discomfort)/len(discomfort))*10)**2)

    df = pd.DataFrame([idfFile,electricity_cost,discomfort_cost,obj,ph_mag,ph_dur,sb_mag,pc1_mag,pc1_dur,su1_mag,pc2_mag,pc2_dur,su2_mag])
    df = df.transpose()

    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    df.to_csv(output_dir + '\\output.csv', mode='w', index = False, header=None)
    print(occupied_time)
    
    


In [ ]:
df_temp = pd.DataFrame([time_list,electricity]).transpose()
df_temp.to_csv("electricity_cost_array.csv")

In [ ]:
df_TR = pd.read_csv("electricity_cost_array_TR.csv", names=["time","cost"])
df_opt = pd.read_csv("electricity_cost_array_opt.csv", names=["time","cost"])
# plt.plot(df_TR["cost"])
# plt.plot(df_opt["cost"])
df_diff = df_TR["cost"] - df_opt["cost"]
df_diff.min()

# Create all results

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
scenarios_info.info()

energy = [[ ] for _ in range(8)]
discomfort = [[ ] for _ in range(8)]

for i, row in scenarios_info.iterrows():
    df = pd.read_csv("Results/Scenario_{}/output.csv".format(row.ID), names=["idfFile","electricity_cost","discomfort_cost","obj","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
    if row.description == "GA":
        print(row.ID)
        energy[0].append(df["electricity_cost"].values[0])
        discomfort[0].append(df["discomfort_cost"].values[0])
    elif row.description == "no_dlc":
        energy[1].append(df["electricity_cost"].values[0])
        discomfort[1].append(df["discomfort_cost"].values[0])
    elif row.description == "dlc_1":
        energy[2].append(df["electricity_cost"].values[0])
        discomfort[2].append(df["discomfort_cost"].values[0])
    elif row.description == "dlc_2":
        energy[3].append(df["electricity_cost"].values[0])
        discomfort[3].append(df["discomfort_cost"].values[0])
    elif row.description == "dlc_3":
        energy[4].append(df["electricity_cost"].values[0])
        discomfort[4].append(df["discomfort_cost"].values[0])

    elif row.description == "dlc_4":
        energy[5].append(df["electricity_cost"].values[0])
        discomfort[5].append(df["discomfort_cost"].values[0])
    elif row.description == "dlc_5":
        energy[6].append(df["electricity_cost"].values[0])
        discomfort[6].append(df["discomfort_cost"].values[0])
    elif row.description == "dlc_6":
        energy[7].append(df["electricity_cost"].values[0])
        discomfort[7].append(df["discomfort_cost"].values[0])


energy_arr = np.array(energy)
discomfort_arr = np.array(discomfort)
combined_array = np.transpose(np.concatenate((energy_arr, discomfort_arr), axis=0))
combined_array.shape
df_all = pd.DataFrame(combined_array, columns=["energy_GA","energy_nodlc","energy_dlc1","energy_dlc2","energy_dlc3","energy_dlc4","energy_dlc5","energy_dlc6", "discomfort_GA","discomfort_nodlc","discomfort_dlc1","discomfort_dlc2","discomfort_dlc3","discomfort_dlc4","discomfort_dlc5","discomfort_dlc6"])
df_all.to_csv("all_results.csv")
print("done")

In [ ]:
parameters = []
for i in range(1,37):
    with open('Results\Scenario_{}\opt_results.pkl'.format(i), 'rb') as f:
        results = pickle.load(f)
        parameters.append(results.x)
        
df = pd.DataFrame(parameters, columns=["ph_mag","ph_dur","sb_mag", "pc1_mag", "pc1_dur", "su1_mag", "pc2_mag", "pc2_dur", "su2_mag"])
df.to_csv("optimization_parameters.csv")

# Boxplot all results

In [ ]:
df = pd.read_excel("scenarios_all_results_v2.xlsx")
df.info()
df_energy_dlc = df.filter(regex="energy_dlc")

# x_labels = ["ge/ns/1","ne/ns/1","pe/ns/1","ge/us/1","ne/us/1","pe/us/1","ge/os/1","ne/os/1","pe/os/1",
#             "ge/ns/2","ne/ns/2","pe/ns/2","ge/us/2","ne/us/2","pe/us/2","ge/os/2","ne/os/2","pe/os/2",
#             "ge/ns/3","ne/ns/3","pe/ns/3","ge/us/3","ne/us/3","pe/us/3","ge/os/3","ne/os/3","pe/os/3",
#             "ge/ns/4","ne/ns/4","pe/ns/4","ge/us/4","ne/us/4","pe/us/4","ge/os/4","ne/os/4","pe/os/4"]
x_labels = ["he/ns/1","ae/ns/1","pe/ns/1","he/us/1","ae/us/1","pe/us/1","he/os/1","ae/os/1","pe/os/1",
            "he/ns/2","ae/ns/2","pe/ns/2","he/us/2","ae/us/2","pe/us/2","he/os/2","ae/os/2","pe/os/2",
            "he/ns/3","ae/ns/3","pe/ns/3","he/us/3","ae/us/3","pe/us/3","he/os/3","ae/os/3","pe/os/3",
            "he/ns/4","ae/ns/4","pe/ns/4","he/us/4","ae/us/4","pe/us/4","he/os/4","ae/os/4","pe/os/4"]

fig, ax = plt.subplots(figsize=(12,8))
a = np.arange(0,36)
sns.scatterplot(x = a, y = df["energy_nodlc"], label= "no DLC")
sns.scatterplot(x = a, y = df["energy_GA"], label="optimal")
df_energy_dlc_t = df_energy_dlc.transpose()
sns.boxplot(df_energy_dlc_t, color="lightcoral").set(xticklabels=x_labels,ylabel="Annual cost [$]", xlabel="Scenarios - envelopes, HVAC sizes, occupant preferences")
plt.xticks(rotation = 60)

ax.legend(["No DLC", "Optimal", "Baseline DLCs"], loc="upper right")
plt.savefig("boxplots_cost.png", dpi=300)

In [ ]:
df = pd.read_excel("scenarios_all_results_v2.xlsx")
tot_occupied = 10955/4
df_disc_dlc = df.filter(regex="discomfort_dlc") / tot_occupied

# x_labels = ["ge/ns/1","ne/ns/1","pe/ns/1","ge/us/1","ne/us/1","pe/us/1","ge/os/1","ne/os/1","pe/os/1",
#             "ge/ns/2","ne/ns/2","pe/ns/2","ge/us/2","ne/us/2","pe/us/2","ge/os/2","ne/os/2","pe/os/2",
#             "ge/ns/3","ne/ns/3","pe/ns/3","ge/us/3","ne/us/3","pe/us/3","ge/os/3","ne/os/3","pe/os/3",
#             "ge/ns/4","ne/ns/4","pe/ns/4","ge/us/4","ne/us/4","pe/us/4","ge/os/4","ne/os/4","pe/os/4"]
x_labels = ["he/ns/1","ae/ns/1","pe/ns/1","he/us/1","ae/us/1","pe/us/1","he/os/1","ae/os/1","pe/os/1",
            "he/ns/2","ae/ns/2","pe/ns/2","he/us/2","ae/us/2","pe/us/2","he/os/2","ae/os/2","pe/os/2",
            "he/ns/3","ae/ns/3","pe/ns/3","he/us/3","ae/us/3","pe/us/3","he/os/3","ae/os/3","pe/os/3",
            "he/ns/4","ae/ns/4","pe/ns/4","he/us/4","ae/us/4","pe/us/4","he/os/4","ae/os/4","pe/os/4"]

fig, ax = plt.subplots(figsize=(12,8))
a = np.arange(0,36)
sns.scatterplot(x = a, y = df["discomfort_nodlc"]/ tot_occupied, label= "no DLC")
sns.scatterplot(x = a, y = df["discomfort_GA"]/ tot_occupied, label="optimal")
df_disc_dlc_t = df_disc_dlc.transpose()
sns.boxplot(df_disc_dlc_t, color="lightcoral").set(xticklabels=x_labels,ylabel="$f_{discomfort}$", xlabel="Scenarios - envelopes, HVAC sizes, occupant preferences")
plt.xticks(rotation = 60)
plt.ylabel("$f_{discomfort}$", fontsize=15)
plt.xlabel("Scenarios - envelopes, HVAC sizes, occupant preferences", fontsize=12)


ax.legend(["No DLC", "Optimal", "Baseline DLCs"], loc="upper right")
plt.savefig("boxplots_discomfort.png", dpi=300)

# Setpoint profiles

In [ ]:
for case in range(1,37):  #,37
    with open('Results\Scenario_{}\opt_results.pkl'.format(case), 'rb') as f:
        results = pickle.load(f)

    day_of_week = range(1,8)
    time_of_day = np.linspace(0,24,96)
    month = 1
    
    a = 8.7  #c/kWh  off_peak_price
    b = 12.2  #c/kWh  mid_peak_price
    c = 18.2  #c/kWh  on_peak_price
    time_rates = np.array([0, 7, 11, 17, 19, 24])
    winter_rates = np.array([a, c, b, c, a, a])
    summer_rates = np.array([a, b, c, b, a, a])
    winter_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:c ,8:c ,9:c ,10:c ,11:b ,12:b ,13:b ,14:b ,15:b ,16:b ,17:c ,18:c ,19:a ,20:a ,21:a ,22:a ,23:a }
    summer_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:b ,8:b ,9:b ,10:b ,11:c ,12:c ,13:c ,14:c ,15:c ,16:c ,17:b ,18:b ,19:a ,20:a ,21:a ,22:a ,23:a }


    initializer_coeff_precond = 2 # preconditioning parameter multipliers
    initializer_coeff_sb_su = 2 # setup/setback parameter multipliers

    ph_mag = results.x[0]
    ph_dur = results.x[1]
    sb_mag = results.x[2]

    pc1_mag = results.x[3]
    pc1_dur = results.x[4]
    su1_mag = results.x[5]

    pc2_mag = results.x[6]
    pc2_dur = results.x[7]
    su2_mag = results.x[8]

    store_hsp = []
    store_csp = []

    for i in range(1,2):
        for j in range(96):
            hsp = 18
            if (day_of_week[i] > 1):
                if (time_of_day[j] > 7 and time_of_day[j] < 17):
                    hsp = 21
            if ((month > 10) or (month < 5)) and (day_of_week[i] > 1 and day_of_week[i] < 7):
                if (time_of_day[j] > 7 - ph_dur) and (time_of_day[j] < 7): # preheat winter off-peak
                    hsp = 18 + ph_mag
                if (time_of_day[j] >= 7) and (time_of_day[j] < 11): # setback winter on-peak
                    hsp = 21 - sb_mag
            store_hsp.append(hsp)

    

    month = 5

    for i in range(1,2):
        for j in range(96):
            csp = 27
            if (day_of_week[i] > 1):
                if (time_of_day[j] > 7 and time_of_day[j] < 17):
                    csp = 24
            if ((month > 4) and (month < 11)) and (day_of_week[i] > 1 and day_of_week[i] < 7):
                if (time_of_day[j] > 7 - pc1_dur) and (time_of_day[j] < 7): # precool summer off-peak
                    csp = 27 - pc1_mag
                if (time_of_day[j] >= 7) and (time_of_day[j] < 11 - pc2_dur): # setup summer mid-peak
                    csp = 24 + su1_mag
                if (time_of_day[j] >= 11 - pc2_dur) and (time_of_day[j] < 11): # precool summer mid-peak
                    csp = 24 - pc2_mag
                if (time_of_day[j] >= 11) and (time_of_day[j] <= 17): # setup summer on-peak
                    csp = 24 + su2_mag

            store_csp.append(csp)        
    
    store_hsp = pd.DataFrame(store_hsp)
    fig, ax = plt.subplots(2, 1, figsize=(7,5))
    ax[0].step(time_of_day,store_hsp,where='post', label="Setpoints");
    ax2 = ax[0].twinx()
    ax2.step(time_rates,winter_rates,where='post', color='darkorange', linestyle="--", label="Rates");
    ax[0].set(xlabel='time of day (h)', ylabel='$HSP \, (^{\circ}C)$')
    ax2.set_ylabel("Electricity rates c/kWh", color='darkorange')
    ax2.set_ylim([8,19])
    ax[0].set_xticks(np.arange(0, 25, 1))
    ax[0].set_yticks(np.arange(18, 25, 1))
    ax[0].grid()
    lines, labels = ax[0].get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax[0].legend(lines + lines2, labels + labels2)
    plt.tight_layout()
    
    store_csp = pd.DataFrame(store_csp)
    ax[1].step(time_of_day,store_csp,where='post');
    ax2 = ax[1].twinx()
    ax2.step(time_rates,summer_rates,where='post', color='darkorange', linestyle="--");
    ax[1].set_xticks(np.arange(0, 25, 1))
    ax[1].set_yticks(np.arange(21, 28, 1))
    ax[1].set(xlabel='time of day (h)', ylabel='$CSP \, (^{\circ}C)$')
    ax2.set_ylabel("Electricity rates c/kWh", color='darkorange')
    ax2.set_ylim([8,19])
    ax[1].grid()
    plt.tight_layout()
    plt.savefig('Results\\Scenario_{}\\setpoints.jpg'.format(case), dpi=300, bbox_inches='tight');

In [94]:
a = 8.7  #c/kWh  off_peak_price
b = 12.2  #c/kWh  mid_peak_price
c = 18.2  #c/kWh  on_peak_price
time_rates = np.array([0, 7, 11, 17, 19, 24])
winter_rates = np.array([a, c, b, c, a, a])
summer_rates = np.array([a, b, c, b, a, a])

all_hsp = []
all_csp = []
fig, ax = plt.subplots(2,1)
for case in range(1,37):  #,37
    with open('Results\Scenario_{}\opt_results.pkl'.format(case), 'rb') as f:
        results = pickle.load(f)

    day_of_week = range(1,8)
    time_of_day = np.linspace(0,24,96)
    month = 1
    
    a = 8.7  #c/kWh  off_peak_price
    b = 12.2  #c/kWh  mid_peak_price
    c = 18.2  #c/kWh  on_peak_price
    time_rates = np.array([0, 7, 11, 17, 19, 24])
    winter_rates = np.array([a, c, b, c, a, a])
    summer_rates = np.array([a, b, c, b, a, a])
    winter_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:c ,8:c ,9:c ,10:c ,11:b ,12:b ,13:b ,14:b ,15:b ,16:b ,17:c ,18:c ,19:a ,20:a ,21:a ,22:a ,23:a }
    summer_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:b ,8:b ,9:b ,10:b ,11:c ,12:c ,13:c ,14:c ,15:c ,16:c ,17:b ,18:b ,19:a ,20:a ,21:a ,22:a ,23:a }


    initializer_coeff_precond = 2 # preconditioning parameter multipliers
    initializer_coeff_sb_su = 2 # setup/setback parameter multipliers

    ph_mag = results.x[0]
    ph_dur = results.x[1]
    sb_mag = results.x[2]

    pc1_mag = results.x[3]
    pc1_dur = results.x[4]
    su1_mag = results.x[5]

    pc2_mag = results.x[6]
    pc2_dur = results.x[7]
    su2_mag = results.x[8]

    store_hsp = []
    store_csp = []

    for i in range(1,2):
        for j in range(96):
            hsp = 18
            if (day_of_week[i] > 1):
                if (time_of_day[j] > 7 and time_of_day[j] < 17):
                    hsp = 21
            if ((month > 10) or (month < 5)) and (day_of_week[i] > 1 and day_of_week[i] < 7):
                if (time_of_day[j] > 7 - ph_dur) and (time_of_day[j] < 7): # preheat winter off-peak
                    hsp = 18 + ph_mag
                if (time_of_day[j] >= 7) and (time_of_day[j] < 11): # setback winter on-peak
                    hsp = 21 - sb_mag
            store_hsp.append(hsp)

    

    month = 5

    for i in range(1,2):
        for j in range(96):
            csp = 27
            if (day_of_week[i] > 1):
                if (time_of_day[j] > 7 and time_of_day[j] < 17):
                    csp = 24
            if ((month > 4) and (month < 11)) and (day_of_week[i] > 1 and day_of_week[i] < 7):
                if (time_of_day[j] > 7 - pc1_dur) and (time_of_day[j] < 7): # precool summer off-peak
                    csp = 27 - pc1_mag
                if (time_of_day[j] >= 7) and (time_of_day[j] < 11 - pc2_dur): # setup summer mid-peak
                    csp = 24 + su1_mag
                if (time_of_day[j] >= 11 - pc2_dur) and (time_of_day[j] < 11): # precool summer mid-peak
                    csp = 24 - pc2_mag
                if (time_of_day[j] >= 11) and (time_of_day[j] <= 17): # setup summer on-peak
                    csp = 24 + su2_mag

            store_csp.append(csp)        
    
    all_hsp.append(store_hsp)
    all_csp.append(store_csp)

all_hsp = np.array(all_hsp)
all_csp = np.array(all_csp)
x = np.arange(0,24,0.25)
ax[0].step(x,np.mean(all_hsp, axis=0),zorder=37, c="r", lw=2)
ax[0].step(x,np.transpose(all_hsp), color="lightgrey")
ax[0].legend(["Mean", "Single scenario"])
ax[0].set_ylabel('$HSP \, (^{\circ}C)$')
ax[0].set_xticks(np.arange(0, 25, 1))
ax[0].set_yticks(np.arange(18, 25, 1))

# ax2 = ax[0].twinx()
# ax2.step(time_rates,winter_rates,where='post', color='darkorange', linestyle="--", label="Rates");
# ax2.set_ylabel("Electricity rates c/kWh", color='darkorange')
# ax2.set_ylim([8,19])

ax[1].step(x,np.mean(all_csp, axis=0), zorder=37, c="r", lw=2)
ax[1].step(x,np.transpose(all_csp), color="lightgrey")
ax[1].legend(["Mean", "Single scenario"])
ax[1].set_xlabel("Time of day (h)")
ax[1].set_ylabel('$CSP \, (^{\circ}C)$')
ax[1].set_xticks(np.arange(0, 25, 1))
ax[1].set_yticks(np.arange(21, 28, 1))

# ax2 = ax[1].twinx()
# ax2.step(time_rates,summer_rates,where='post', color='darkorange', linestyle="--", label="Rates");
# ax2.set_ylabel("Electricity rates c/kWh", color='darkorange')
# ax2.set_ylim([8,19])
plt.savefig('Aggregated_setpoints.jpg', dpi=300)



In [62]:
len(np.arange(0,24,0.25))

96

# Evolution of cost & parameters

In [ ]:
case_no = 7
var_iter = 7
var_pop = 8*9
df = pd.read_csv("Results/Scenario_{}/document.csv".format(case_no), names=["elct","discomfort","objective","ph_mag","ph_dur","sb_mag", "pc1_mag", "pc1_dur", "su1_mag", "pc2_mag", "pc2_dur", "su2_mag"])

fig, ax = plt.subplots(3,1, figsize=(7,6))

pop_list = []
for i in range(var_iter):
    pop_list.append(df["objective"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[0]).set(xlabel="Iterations", ylabel="Objective")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["elct"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[1]).set(xlabel="Iterations", ylabel="Electricity cost [$]")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["discomfort"][i*var_pop:(i+1)*var_pop].values /2504)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[2]).set(xlabel="Iterations", ylabel="Discomfort fraction")
plt.savefig("Iterations_good env - oversized HVAC-20 26.png", dpi=300)

In [ ]:
case_no = 4
var_iter = 7
var_pop = 8*9
df = pd.read_csv("Results/Scenario_{}/document.csv".format(case_no), names=["elct","discomfort","objective","ph_mag","ph_dur","sb_mag", "pc1_mag", "pc1_dur", "su1_mag", "pc2_mag", "pc2_dur", "su2_mag"])

fig, ax = plt.subplots(3,1)

pop_list = []
for i in range(var_iter):
    pop_list.append(df["ph_dur"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[0]).set(xlabel="Iterations", ylabel="ph_dur")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["ph_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[1]).set(xlabel="Iterations", ylabel="ph_mag")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["sb_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[2]).set(xlabel="Iterations", ylabel="sb_mag")

In [ ]:
case_no = 1
var_iter = 7
var_pop = 8*9
df = pd.read_csv("Results/Scenario_{}/document.csv".format(case_no), names=["elct","discomfort","objective","ph_mag","ph_dur","sb_mag", "pc1_mag", "pc1_dur", "su1_mag", "pc2_mag", "pc2_dur", "su2_mag"])

fig, ax = plt.subplots(3,2)

pop_list = []
for i in range(var_iter):
    pop_list.append(df["pc1_dur"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[0,0]).set(xlabel="Iterations", ylabel="pc1_dur")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["pc1_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[1,0]).set(xlabel="Iterations", ylabel="pc1_mag")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["su1_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[2,0]).set(xlabel="Iterations", ylabel="su1_mag")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["pc2_dur"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[0,1]).set(xlabel="Iterations", ylabel="pc2_dur")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["pc2_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[1,1]).set(xlabel="Iterations", ylabel="pc2_mag")

pop_list = []
for i in range(var_iter):
    pop_list.append(df["su2_mag"][i*var_pop:(i+1)*var_pop].values)
df_box = pd.DataFrame(pop_list).transpose()
sns.boxplot(df_box, ax=ax[2,1]).set(xlabel="Iterations", ylabel="su2_mag")

# Pareto diagram

In [95]:
case_no = 1
fig, ax = plt.subplots()
for case_no in [1]:  #range(1,4) [1,4,7]
    df = pd.read_csv("Results/Scenario_{}/document.csv".format(case_no), names=["elct","discomfort","objective","ph_mag","ph_dur","sb_mag", "pc1_mag", "pc1_dur", "su1_mag", "pc2_mag", "pc2_dur", "su2_mag"])
    df_all = pd.read_excel("scenarios_all_results_v2.xlsx".format(case_no))
    df_select = df_all[df_all["ID"] == 1]
    
    no_dlc_point = [df_select["energy_nodlc"].values[0], df_select["discomfort_nodlc"].values[0]/2504]
    GA_point = [df_select["energy_GA"].values[0], df_select["discomfort_GA"].values[0]/2504] 
   
    
    sns.scatterplot(x=df["elct"], y=df["discomfort"]/2504, ax=ax).set(ylabel="$f_{discomfort}$", xlabel="Annual cost [$]")
    ax.scatter(no_dlc_point[0], no_dlc_point[1], marker='x', linewidths=3)
    ax.scatter(GA_point[0], GA_point[1], marker=6, linewidths=3)
    
# plt.legend(["Normal", "Undersized", "Oversized"])
# plt.legend(["High env.", "Average env.", "Poor env."])
plt.legend(["Iterations", "No DLC", "Optimal"])
plt.savefig("Pareto Optimality.png", dpi=300)

In [33]:
GA_point

[950.599126329802, 164.75]

# Table of savings

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
elct_cost = []
for case in range(1,37):
    df = pd.read_csv("Results/Scenario_{}/output.csv".format(case), names=["idfFile","electricity_cost","discomfort_cost","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
    elct_cost.append(df["electricity_cost"].values[0])

base_cost = []
for case in range(1,37):
    df = pd.read_csv("basecases/Scenario_{}/output.csv".format(case), names=["idfFile","electricity_cost","discomfort_cost","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
    base_cost.append(df["electricity_cost"].values[0])

df_bars = pd.DataFrame([elct_cost, base_cost]).transpose()
df_bars.columns = ["DLC","basecase"]

df_reset = df_bars.reset_index()
df_melted = df_reset.melt(id_vars='index', var_name='variable', value_name='value')
sns.barplot(x='index', y='value', hue='variable', data=df_melted)

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_opt.xlsx')
discomfort_cost = []
for case in range(1,37):
    df = pd.read_csv("Results/Scenario_{}/output.csv".format(case), names=["idfFile","electricity_cost","discomfort_cost","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
    discomfort_cost.append(df["discomfort_cost"].values[0])
print(discomfort_cost)

base_cost = []
for case in range(1,37):
    df = pd.read_csv("basecases/Scenario_{}/output.csv".format(case), names=["idfFile","electricity_cost","discomfort_cost","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
    base_cost.append(df["discomfort_cost"].values[0])

df_bars = pd.DataFrame([elct_cost, base_cost]).transpose()
df_bars.columns = ["DLC","basecase"]

df_reset = df_bars.reset_index()
df_melted = df_reset.melt(id_vars='index', var_name='variable', value_name='value')
sns.barplot(x='index', y='value', hue='variable', data=df_melted)

In [ ]:
df = pd.read_csv("Results/Scenario_{}/output.csv".format(288), names=["idfFile","electricity_cost","discomfort_cost","ph_mag","ph_dur","sb_mag","pc1_mag","pc1_dur","su1_mag","pc2_mag","pc2_dur","su2_mag"])
df["electricity_cost"]